### Read reviews from Silver

In [0]:
from pyspark.sql import functions as F

reviews_df = (
    spark.table("olist_ecommerce.silver.reviews")
    .filter(F.col("review_comment_message").isNotNull())
)

### Define sentiment prompt

In [0]:
SENTIMENT_PROMPT = """
Classify the sentiment of the following customer review.
Respond with ONLY one word from:
positive, neutral, negative.

Review:
"""

### Run sentiment using Mosaic AI

In [0]:
sentiment_df = reviews_df.withColumn(
    "sentiment",
    F.expr(f"""
        ai_query(
            "databricks-meta-llama-3-3-70b-instruct",
            concat('{SENTIMENT_PROMPT}', review_comment_message)
        )
    """)
)

### Clean model output

In [0]:
sentiment_df = sentiment_df.withColumn(
    "sentiment",
    F.lower(F.trim(F.col("sentiment")))
)

### Select Gold-ready columns

In [0]:
gold_sentiment_df = sentiment_df.select(
    "review_id",
    "order_id",
    "review_score",
    "sentiment"
)

### Write to Gold

In [0]:
gold_sentiment_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("olist_ecommerce.gold.review_sentiment")